In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [13]:
folder_path = "/Users/oas/Downloads/benchmarking_results"
# folder_path = "/Users/oas/Documents/VOD/vod/benchmarking_results"
files = [
    os.path.join(folder_path, file)
    for file in os.listdir(folder_path)
    if os.path.isfile(os.path.join(folder_path, file))
]
files.sort()  # Sort the files by name
newest_file = files[-1]  # Pick the newest
print(newest_file)

/Users/oas/Downloads/benchmarking_results/2024-02-19-12-25-45.csv


In [14]:
df_results = pd.read_csv(newest_file)
df_results.shape

(15, 16)

In [23]:
df_results

,Unnamed: 0,Dataset,Index,Index parameters.,Server startup speed (s),Index build speed (s),Search speed avg. (ms),Search speed p95 (ms),Recall avg,Recall@1,Recall@10,Recall@100,ingesting_max,ingesting_mean,benchmarking_max,benchmarking_mean,index_prov
0,0,gist-960-euclidean,"index: faiss IVF, n_partition=100, None, DOT","IVF, n_partition=100, None, DOT",29.677277,12.114791,490.752727,530.608323,0.000433,0.000333,0.000667,0.003000,4149.248,1230.429600,4151.296,4151.296000,faiss
1,1,gist-960-euclidean,"index: faiss IVF, n_partition=200, None, DOT","IVF, n_partition=200, None, DOT",28.091996,14.724321,383.251042,522.979428,0.000503,0.000333,0.000667,0.003667,1025.024,504.531000,4151.296,4151.296000,faiss
2,2,gist-960-euclidean,"index: faiss IVF, n_partition=400, None, DOT","IVF, n_partition=400, None, DOT",24.369994,20.912197,309.938170,516.331521,0.000550,0.000333,0.001000,0.004333,913.400,475.575000,4152.320,4152.320000,faiss
3,3,gist-960-euclidean,"index: faiss IVF, n_partition=800, None, DOT","IVF, n_partition=800, None, DOT",14.435392,35.834797,254.547133,512.055937,0.000757,0.000333,0.001667,0.006667,935.500,481.100000,4154.368,4154.368000,faiss
4,4,gist-960-euclidean,"index: faiss IVF, n_partition=1600, None, DOT","IVF, n_partition=1600, None, DOT",-15.163393,76.522429,213.412361,507.295878,0.000960,0.000333,0.001333,0.009000,2928.640,841.968000,4157.440,4157.440000,faiss
8,8,gist-960-euclidean,"index: faiss IVF, n_partition=800, PQ8, DOT","IVF, n_partition=800, PQ8, DOT",-390.127873,531.124948,179.575678,503.569848,0.001363,0.000000,0.000000,0.002000,414.500,330.633333,502.800,502.643333,faiss
9,9,gist-960-euclidean,"index: faiss IVF, n_partition=1600, PQ8, DOT","IVF, n_partition=1600, PQ8, DOT",-288.399640,478.823034,155.345527,498.282929,0.002580,0.000000,0.000333,0.002667,414.200,330.333333,505.700,505.576364,faiss
10,10,gist-960-euclidean,"index: faiss IVF, n_partition=100, SQ8, DOT","IVF, n_partition=100, SQ8, DOT",157.909800,11.987786,201.531834,533.332461,0.000493,0.000000,0.000667,0.002667,414.300,330.200000,1406.976,1405.995390,faiss
11,11,gist-960-euclidean,"index: faiss IVF, n_partition=200, SQ8, DOT","IVF, n_partition=200, SQ8, DOT",137.454056,17.053442,211.917280,531.105791,0.000580,0.000000,0.000667,0.002667,452.300,315.175000,1406.976,1406.976000,faiss
12,12,gist-960-euclidean,"index: faiss IVF, n_partition=400, SQ8, DOT","IVF, n_partition=400, SQ8, DOT",113.355539,30.161418,208.207856,528.579564,0.000603,0.000000,0.001000,0.003333,453.800,319.525000,1408.000,1408.000000,faiss


In [16]:
df_results = df_results.query("Index != 'None'")  # ignore failed runs
df_results = df_results.query("benchmarking_mean != -1")  # ignore failed runs

In [17]:
df_results["index_prov"] = df_results.Index.str.split(" ").apply(lambda l: l[1])

In [18]:
df_results = df_results.dropna()

In [19]:
df_results.columns

Index(['Unnamed: 0', 'Dataset', 'Index', 'Index parameters.',
       'Server startup speed (s)', 'Index build speed (s)',
       'Search speed avg. (ms)', 'Search speed p95 (ms)', 'Recall avg',
       'Recall@1', 'Recall@10', 'Recall@100', 'ingesting_max',
       'ingesting_mean', 'benchmarking_max', 'benchmarking_mean',
       'index_prov'],
      dtype='object')

In [20]:
df_results.shape

(12, 17)

In [21]:
# df_results[df_results.isna()]

In [22]:
import plotly.express as px
import plotly.graph_objects as go

# Your existing DataFrame and plot
fig = px.scatter(
    df_results,
    x="Recall avg",
    y="Search speed avg. (ms)",
    hover_data=["Index","ingesting_max","benchmarking_mean"],
    color="index_prov",
    size=df_results["benchmarking_mean"],
)

# Adding a similar scatter plot but with semi-transparency and different size mapping
fig.add_trace(
    go.Scatter(
        x=df_results["Recall avg"],
        y=df_results["Search speed avg. (ms)"],
        mode="markers",
        marker=dict(
            size=df_results["ingesting_max"] / 25,  # Size based on 'ingesting_max' column
            opacity=0.1,  # Semi-transparent markers
            # color=fig.data[0].marker.color,  # Use the same color as the first plot
            # line=dict(color="MediumPurple", width=2),
        ),
        hoverinfo="text",  # You can customize hover info as needed
        name="Ingesting max",
    )
)

fig.show()